# Подсчет безлунного ночного времени 

Для ответа рецензенту статьи про моделирование.

5 марта 2018 г.

Данные для расчетов безлунного времени взяты с сайта 
http://www.abakbot.ru/online-6/29-rise-and-set-sun-moon
для населенного пункта 
Irkutsk, Irkutsk Oblast, Russia.

Сайт выдает данные:
День, Восход Солнца, Заход Солнца, Восход Луны, Заход Луны.
Время местное для указанного населенного пункта.

Координаты и высота над уровнем моря населенного пункта Иркутск незначительно отличаются от условий точки старта на Байкале. Отличие по временам - на 3-5 минут.

In [1]:
!date
!pwd

Пн мар  5 20:07:25 MSK 2018
/home/sphere/Papers/2017_Model/Luna


In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.gridspec as gridspec
import numpy as np

from datetime import timedelta

In [3]:
# считать данные, заменив None на NaN
na_values = ['None', 'N/A']
data = pd.read_csv('./2013_01-04_Irkutsk.csv', na_values=na_values, index_col=False, sep='\t')

In [4]:
data.head(3)

,Day,Sunrise,Sunset,Moonrise,Moonset
0,1/1/2013,10:12,18:00,22:32,11:38
1,2/1/2013,10:12,18:01,23:43,12:00
2,3/1/2013,10:11,18:02,NaN,12:21


От времени восхода и заката Солнца отступаем по 90 минут, от Луны - не отступаем.

In [5]:
data['SunriseTime'] = pd.to_datetime(data.Day + ' ' + data.Sunrise, dayfirst = True) - timedelta(minutes=90)
data['SunsetTime']  = pd.to_datetime(data.Day + ' ' + data.Sunset,  dayfirst = True) + timedelta(minutes=90)

data['MoonriseTime'] = pd.to_datetime(data.Day + ' ' + data.Moonrise, dayfirst = True)
data['MoonsetTime']  = pd.to_datetime(data.Day + ' ' + data.Moonset,  dayfirst = True) 

In [6]:
params = ['Day','SunriseTime','MoonriseTime','SunsetTime','MoonsetTime']
data = data[params]
data.head(3)

,Day,SunriseTime,MoonriseTime,SunsetTime,MoonsetTime
0,1/1/2013,2013-01-01 08:42:00,2013-01-01 22:32:00,2013-01-01 19:30:00,2013-01-01 11:38:00
1,2/1/2013,2013-01-02 08:42:00,2013-01-02 23:43:00,2013-01-02 19:31:00,2013-01-02 12:00:00
2,3/1/2013,2013-01-03 08:41:00,NaT,2013-01-03 19:32:00,2013-01-03 12:21:00


In [7]:
# count stop and start times
for i in range(data.shape[0]):
    SR = data.SunriseTime[i]
    SS = data.SunsetTime[i]
    MR = data.MoonriseTime[i]
    MS = data.MoonsetTime[i]
    flag = 0
    
    stop  = SR    
    if str( MR ) == 'NaT': 
        #stop = pd.to_datetime('NaT')
        flag = 1
        #print('MR NaT')
    else:
        if (MR < SR) or (MR > SS):
            stop = MR
                
    start = SS
    if str( MS ) == 'NaT':
        start = pd.to_datetime('NaT')
        flag = 2
        #print('MS NaT')
    else:
        if (MS < SR) or (SS < MS):
                start = MS
        else: 
            if MS < MR and MR < SS:
                start = pd.to_datetime('NaT')
                flag = 33
        
    data.loc[i, 'Start'] = start
    data.loc[i, 'Stop']  = stop
   
    delta = stop - start
    #print(start,'-', stop,'=', delta, 'fl', flag)

In [8]:
# count delta times
for i in range(data.shape[0]-1):
    SR = data.SunriseTime[i]
    SS = data.SunsetTime[i]
    MR = data.MoonriseTime[i]
    MS = data.MoonsetTime[i]
    stop   = data.Stop[i]
    start  = data.Start[i]
    startnext = data.Start[i+1]
    stopnext  = data.Stop[i+1]
    MRnext    = data.MoonriseTime[i+1]
    
    deltanull = timedelta(seconds=0)

    if str( start ) == 'NaT': 
        start = startnext
        
    if start <= stop:
        delta = stop - start
    else:
        if startnext <= stopnext  or  str(start) == 'NaT':
            delta = deltanull
        else:
            delta = stopnext - start
            
    data.loc[i, 'Delta'] = delta   
    #print(start,'-', stop,'=', delta)

data.Delta = data.Delta.fillna(deltanull)

In [9]:
data['Month'] = data['SunriseTime'].apply(lambda ts: ts.month)

In [10]:
data

,Day,SunriseTime,MoonriseTime,SunsetTime,MoonsetTime,Start,Stop,Delta,Month
0,1/1/2013,2013-01-01 08:42:00,2013-01-01 22:32:00,2013-01-01 19:30:00,2013-01-01 11:38:00,2013-01-01 19:30:00,2013-01-01 22:32:00,03:02:00,1
1,2/1/2013,2013-01-02 08:42:00,2013-01-02 23:43:00,2013-01-02 19:31:00,2013-01-02 12:00:00,2013-01-02 19:31:00,2013-01-02 23:43:00,04:12:00,1
2,3/1/2013,2013-01-03 08:41:00,NaT,2013-01-03 19:32:00,2013-01-03 12:21:00,2013-01-03 19:32:00,2013-01-03 08:41:00,05:22:00,1
3,4/1/2013,2013-01-04 08:41:00,2013-01-04 00:54:00,2013-01-04 19:33:00,2013-01-04 12:43:00,2013-01-04 19:33:00,2013-01-04 00:54:00,06:36:00,1
4,5/1/2013,2013-01-05 08:41:00,2013-01-05 02:09:00,2013-01-05 19:35:00,2013-01-05 13:06:00,2013-01-05 19:35:00,2013-01-05 02:09:00,07:49:00,1
5,6/1/2013,2013-01-06 08:40:00,2013-01-06 03:24:00,2013-01-06 19:36:00,2013-01-06 13:32:00,2013-01-06 19:36:00,2013-01-06 03:24:00,09:07:00,1
6,7/1/2013,2013-01-07 08:40:00,2013-01-07 04:43:00,2013-01-07 19:37:00,2013-01-07 14:04:00,2013-01-07 19:37:00,2013-01-07 04:43:00,10:22:00,1
7,8/1/2013,2013-01-08 08:39:00,2013-01-08 05:59:00,2013-01-08 19:39:00,2013-01-08 14:46:00,2013-01-08 19:39:00,2013-01-08 05:59:00,11:34:00,1
8,9/1/2013,2013-01-09 08:39:00,2013-01-09 07:13:00,2013-01-09 19:40:00,2013-01-09 15:39:00,2013-01-09 19:40:00,2013-01-09 07:13:00,12:37:00,1
9,10/1/2013,2013-01-10 08:38:00,2013-01-10 08:17:00,2013-01-10 19:41:00,2013-01-10 16:42:00,2013-01-10 19:41:00,2013-01-10 08:17:00,12:57:00,1


# Results

Посчитаем сумму безлунных периодов по месяцам

In [11]:
# moonlesstime sum
moonless_time = data.groupby('Month').Delta.sum()
print(moonless_time)

Month
1   7 days 14:51:00
2   6 days 10:21:00
3   5 days 20:48:00
4   4 days 12:45:00
Name: Delta, dtype: timedelta64[ns]


Посчитаем сумму безлунных периодов по месяцам, учитывая только периоды длительностью больше одного часа.

In [12]:
# moonlesstime sum for periods more than 60 min
moonless_time = data[data.Delta > timedelta(minutes=60)].groupby('Month').Delta.sum()
print(moonless_time)

Month
1   7 days 13:44:00
2   6 days 08:48:00
3   5 days 19:52:00
4   4 days 11:03:00
Name: Delta, dtype: timedelta64[ns]


Полная таблица длительностей на каждый день

In [13]:
data[:59][['Day','Start','Stop', 'Delta']]

,Day,Start,Stop,Delta
0,1/1/2013,2013-01-01 19:30:00,2013-01-01 22:32:00,03:02:00
1,2/1/2013,2013-01-02 19:31:00,2013-01-02 23:43:00,04:12:00
2,3/1/2013,2013-01-03 19:32:00,2013-01-03 08:41:00,05:22:00
3,4/1/2013,2013-01-04 19:33:00,2013-01-04 00:54:00,06:36:00
4,5/1/2013,2013-01-05 19:35:00,2013-01-05 02:09:00,07:49:00
5,6/1/2013,2013-01-06 19:36:00,2013-01-06 03:24:00,09:07:00
6,7/1/2013,2013-01-07 19:37:00,2013-01-07 04:43:00,10:22:00
7,8/1/2013,2013-01-08 19:39:00,2013-01-08 05:59:00,11:34:00
8,9/1/2013,2013-01-09 19:40:00,2013-01-09 07:13:00,12:37:00
9,10/1/2013,2013-01-10 19:41:00,2013-01-10 08:17:00,12:57:00
